In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# upload exported cities.csv file from WeatherPy.ipynb as dataframe
cities_df = pd.read_csv("../Output_data/cities.csv")
del cities_df["Unnamed: 0"]
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Comé,40,BJ,1586651214,79,6.41,1.88,84.20,5.82
1,Ulladulla,0,AU,1586651215,39,-35.35,150.47,64.40,12.75
2,Mpika,81,ZM,1586651216,63,-11.83,31.45,62.24,8.32
3,Kavaratti,4,IN,1586651080,69,10.57,72.64,84.04,4.61
4,Saint Anthony,90,US,1586651218,26,45.02,-93.22,59.00,9.17


In [3]:
# configure gmaps
fig = gmaps.figure()
gmaps.configure(api_key = g_key)

# drop NA values
cities_df = cities_df.dropna()

# set up variables
locations = cities_df[["Lat", "Lng"]].astype(float)
humidity = cities_df["Humidity"].astype(float)

# create heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 10, point_radius = .5)

fig.add_layer(heat_layer)

# display map
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# create new dataframe with ideal weather
ideal_df = cities_df

# max temp between 70 & 80 degrees, wind speed below 10 MPH, cloudiness = 0
ideal_df = ideal_df.loc[(ideal_df["Max Temp"] <= 80) & (ideal_df["Max Temp"] >= 70)]
ideal_df = ideal_df.loc[(ideal_df["Wind Speed"] < 10)]
ideal_df = ideal_df.loc[(ideal_df["Cloudiness"] == 0)]

# drop NA values and show header
ideal_df = ideal_df.dropna()
ideal_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
116,Igatpuri,0,IN,1586651338,24,19.7,73.55,74.70,1.43
220,Kānt,0,IN,1586651451,25,27.8,79.80,77.38,6.40
580,Tessalit,0,ML,1586651708,13,20.2,1.01,77.65,2.35


In [5]:
# create hotel_df dataframe and add hotel column
hotel_df = pd.DataFrame(ideal_df)
hotel_df["Hotel"] = ""
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel
116,Igatpuri,0,IN,1586651338,24,19.7,73.55,74.70,1.43,
220,Kānt,0,IN,1586651451,25,27.8,79.80,77.38,6.40,
580,Tessalit,0,ML,1586651708,13,20.2,1.01,77.65,2.35,


In [6]:
# parameters of Google API search, minus location which we'll retrieve in for loop
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# for loop to retrieve lat/lng coordinates
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add coordinates to 'location' search parameter
    params["location"] = f"{lat},{lng}"
    
    # Google API base search url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # call API with new parameters
    response = requests.get(base_url, params=params)
    response_json = response.json()
    
    # go through rows and try to add hotel name information to hotel dataframe
    try:
        hotel_df.loc[index, "Hotel"] = response_json["results"][0]["name"]
        
    # exception for KeyErrors and IndexErrors
    except (KeyError, IndexError):
        print("Missing result...skipping.")
        
# print new dataframe with hotel info
hotel_df

Missing result...skipping.
Missing result...skipping.


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel
116,Igatpuri,0,IN,1586651338,24,19.7,73.55,74.70,1.43,Tropical Retreat Luxury Spa & Resort
220,Kānt,0,IN,1586651451,25,27.8,79.80,77.38,6.40,
580,Tessalit,0,ML,1586651708,13,20.2,1.01,77.65,2.35,


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
marker = gmaps.marker_layer(locations)

hotel_mark = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info)

In [8]:
# add and display hotel markers info
fig.add_layer(marker)
fig.add_layer(hotel_mark)
fig

Figure(layout=FigureLayout(height='420px'))